<a href="https://colab.research.google.com/github/pippa28/GoogleCollab/blob/main/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem**
To predict a number of survival months based on various clinical and demographic features. Additionally, predicting how long a patient will survive.

Importing libraries

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


Loading in the Breast Cancer dataset

In [19]:
data = pd.read_csv('/content/Breast_Cancer.csv')

In [20]:
data.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [21]:
#Check for any missing values
print("\nMissing Values:")
print(data.isnull().sum())

#Fill in any missing values if any
data.fillna(data.mode().iloc[0],inplace=True)


Missing Values:
Age                       0
Race                      0
Marital Status            0
T Stage                   0
N Stage                   0
6th Stage                 0
differentiate             0
Grade                     0
A Stage                   0
Tumor Size                0
Estrogen Status           0
Progesterone Status       0
Regional Node Examined    0
Reginol Node Positive     0
Survival Months           0
Status                    0
dtype: int64


In [22]:
# Encoding categorical variables
categorical_cols = ['race', 'marital_status', 't_stage', 'n_stage', 'a_stage', 'estrogen_status', 'progesterone_status', 'status']
label_encoders = {}
for col in categorical_cols:
    if col in data.columns:  # Ensure column exists
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le  # Store encoders for future decoding

In [23]:
# Scaling numerical variables
numerical_cols = ['age', 'tumor_size', 'regional_node_examined', 'reginol_node_positive']
numerical_cols = [col for col in numerical_cols if col in data.columns]  # Ensure columns exist
if numerical_cols:  # Check if there are numerical columns before scaling
    scaler = StandardScaler()
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])